In [1]:
# --- Install PyCaret 3.x (core) ---
!pip -q install -U pip setuptools wheel
!pip -q install -U pycaret pandas scikit-learn

import sys, platform, numpy as np, pandas as pd
from datetime import datetime
print("Python:", sys.version.split()[0])
print("Pandas:", pd.__version__)
import pycaret
print("PyCaret:", pycaret.__version__)
print("Run time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# GPU check (optional)
import os
gpu = os.popen("nvidia-smi -L").read()
print("GPU:", gpu if gpu else "No GPU visible")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.2 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [8]:
# TITANIC BINARY CLASSIFICATION
import pandas as pd
from pycaret.classification import setup, compare_models, tune_model, blend_models, stack_models, finalize_model, predict_model, save_model, pull

INPUT = "/kaggle/input/titanic" if "/kaggle" in __import__("os").getcwd() else "/content"
train = pd.read_csv('../input/titanic/train.csv') if "titanic" in INPUT else pd.read_csv("/content/train.csv")
test  = pd.read_csv('../input/titanic/test.csv')  if "titanic" in INPUT else pd.read_csv("/content/test.csv")

clf = setup(
    data=train,
    target="Survived",
    session_id=42,
    imputation_type="simple",
    normalize=True,
    fix_imbalance=True,
    use_gpu=True,              # will use GPU where supported
    fold=5,
    verbose=True
)

best = compare_models(n_select=5)
tuned = [tune_model(m, optimize="AUC") for m in best]
blended = blend_models(estimator_list=tuned, optimize="AUC")
stacked = stack_models(estimator_list=tuned, meta_model=None, optimize="AUC")  # let PyCaret pick meta
final  = finalize_model(blended)

print(pull().tail(10))  # leaderboard snapshot

# Predict on test (PyCaret auto-handles missing training-only columns)
pred = predict_model(final, data=test)
sub = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": pred["prediction_label"]})
sub.to_csv("titanic_submission.csv", index=False)
save_model(final, "titanic_best_model")


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because

,Description,Value
0,Session id,42
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 12)"
4,Transformed data shape,"(1036, 14)"
5,Transformed train set shape,"(768, 14)"
6,Transformed test set shape,"(268, 14)"
7,Numeric features,6
8,Categorical features,5
9,Rows with missing values,79.5%


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split req

...

[2025-11-01 00:36:14.892] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-11-01 00:36:14.893] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7760,0.8607,0.6667,0.7273,0.6957,0.5190,0.5202
1,0.8160,0.8772,0.7083,0.7907,0.7473,0.6033,0.6055
2,0.8640,0.8853,0.6667,0.9697,0.7901,0.6946,0.7212
3,0.7823,0.8486,0.5532,0.8125,0.6582,0.5068,0.5269
4,0.7823,0.8459,0.6458,0.7561,0.6966,0.5285,0.5324
Mean,0.8041,0.8635,0.6481,0.8113,0.7176,0.5704,0.5813
Std,0.0331,0.0155,0.0516,0.0844,0.0460,0.0706,0.0765


[2025-11-01 00:36:33.977] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-11-01 00:36:33.979] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-11-01 00:38:06.457] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-11-01 00:38:06.459] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.7760  0.8607  0.6667  0.7273  0.6957  0.5190  0.5202
1       0.8160  0.8772  0.7083  0.7907  0.7473  0.6033  0.6055
2       0.8640  0.8853  0.6667  0.9697  0.7901  0.6946  0.7212
3       0.7823  0.8486  0.5532  0.8125  0.6582  0.5068  0.5269
4       0.7823  0.8459  0.6458  0.7561  0.6966  0.5285  0.5324
Mean    0.8041  0.8635  0.6481  0.8113  0.7176  0.5704  0.5813
Std     0.0331  0.0155  0.0516  0.0844  0.0460  0.0706  

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['PassengerId', 'Pclass', 'Age',
                                              'SibSp', 'Parch', 'Fare'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None...
                                                                     n_estimators=170,
                                            